# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.7/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.0.251/12546/1 Dashboard: http://10.20.0.251:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1057,Patricia,0.8895101968863994,-0.5397910823809065
2000-01-01 00:00:01,1005,Ray,-0.9069024068385223,-0.017392219355852445
2000-01-01 00:00:02,1054,Ray,0.05293096922978413,0.9975888634368186
2000-01-01 00:00:03,978,George,0.7452679715060091,0.455766082290316
2000-01-01 00:00:04,1040,Xavier,0.3068076257350363,0.5387206518634984
2000-01-01 00:00:05,990,Ingrid,0.6315791905756298,0.9275475018748716
2000-01-01 00:00:06,972,Bob,0.5530055406023524,0.9859459792940846
2000-01-01 00:00:07,959,Yvonne,0.10557918570870184,-0.4169608749907596
2000-01-01 00:00:08,990,Victor,0.7615520583150577,-0.18459040346848132


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1012,Laura,-0.6500001746161057,-0.4235273668302837
2000-01-30 00:00:01,1001,Charlie,0.8580539799495084,0.6197269240457133
2000-01-30 00:00:02,966,Laura,0.9719693819465118,-0.834842538995018
2000-01-30 00:00:03,978,Ursula,-0.3479177356603167,0.8563833390365814
2000-01-30 00:00:04,966,Tim,0.3408385694530993,-0.8486942541588132
2000-01-30 00:00:05,1008,Ray,0.9633594453422334,0.05729225200804655
2000-01-30 00:00:06,1044,Victor,0.2656242025469977,-0.34550659455541655
2000-01-30 00:00:07,1013,Charlie,0.5405988655403284,-0.1360770020292965
2000-01-30 00:00:08,1015,Michael,0.6835980765297229,0.7234406024385762


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1057,Patricia,0.889510,-0.539791
1,2000-01-01 00:00:01,1005,Ray,-0.906902,-0.017392
2,2000-01-01 00:00:02,1054,Ray,0.052931,0.997589
3,2000-01-01 00:00:03,978,George,0.745268,0.455766
4,2000-01-01 00:00:04,1040,Xavier,0.306808,0.538721


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1057,Patricia,0.889510,-0.539791
1,2000-01-01 00:00:01,1005,Ray,-0.906902,-0.017392
2,2000-01-01 00:00:02,1054,Ray,0.052931,0.997589
3,2000-01-01 00:00:03,978,George,0.745268,0.455766
4,2000-01-01 00:00:04,1040,Xavier,0.306808,0.538721


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.69 s, sys: 546 ms, total: 5.24 s
Wall time: 3.37 s


name
Alice       0.000820
Bob         0.003158
Charlie    -0.001646
Dan         0.000310
Edith       0.004080
Frank       0.001503
George     -0.001408
Hannah     -0.000676
Ingrid     -0.001354
Jerry       0.000071
Kevin       0.001488
Laura      -0.000870
Michael    -0.001297
Norbert    -0.001090
Oliver     -0.001144
Patricia    0.001037
Quinn      -0.003984
Ray         0.000567
Sarah       0.000849
Tim        -0.000466
Ursula      0.001044
Victor      0.000536
Wendy       0.001481
Xavier      0.001287
Yvonne     -0.000618
Zelda      -0.000381
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.32 s, sys: 216 ms, total: 1.54 s
Wall time: 1.03 s


name
Alice       0.000820
Bob         0.003158
Charlie    -0.001646
Dan         0.000310
Edith       0.004080
Frank       0.001503
George     -0.001408
Hannah     -0.000676
Ingrid     -0.001354
Jerry       0.000071
Kevin       0.001488
Laura      -0.000870
Michael    -0.001297
Norbert    -0.001090
Oliver     -0.001144
Patricia    0.001037
Quinn      -0.003984
Ray         0.000567
Sarah       0.000849
Tim        -0.000466
Ursula      0.001044
Victor      0.000536
Wendy       0.001481
Xavier      0.001287
Yvonne     -0.000618
Zelda      -0.000381
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.31 s, sys: 35 ms, total: 1.34 s
Wall time: 948 ms


name
Alice       0.000820
Bob         0.003158
Charlie    -0.001646
Dan         0.000310
Edith       0.004080
Frank       0.001503
George     -0.001408
Hannah     -0.000676
Ingrid     -0.001354
Jerry       0.000071
Kevin       0.001488
Laura      -0.000870
Michael    -0.001297
Norbert    -0.001090
Oliver     -0.001144
Patricia    0.001037
Quinn      -0.003984
Ray         0.000567
Sarah       0.000849
Tim        -0.000466
Ursula      0.001044
Victor      0.000536
Wendy       0.001481
Xavier      0.001287
Yvonne     -0.000618
Zelda      -0.000381
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html